In [12]:
import __init__ as init
import google_api as g
import requests
import pandas as pd
import numpy as np
import sys
import os

# Notebook Set Up

# Google API set up

In [4]:
# Function to break get_address() response into components
def parse_address(address_json):
    address_components = {
        'housenum_lo': address_json['address_components'][0]['long_name'],
        'street_name': address_json['address_components'][1]['long_name'],
        'neighborhood': address_json['address_components'][2]['long_name'],
        'borough': address_json['address_components'][3]['long_name'],
        'county': address_json['address_components'][4]['long_name'],
        'zip_code': address_json['address_components'][7]['long_name']
    }
    return address_components


# AirBNB

* Take a sample of AirBNB listings weighted proportionally by borough.
* Request addresses from Google, based on lat/long features.
* Send addresses to OpenDataNYC for valuations.


### Sample 10K

### Call Google API - Reverse Geocoder

In [10]:
# try first 1000 rows to avoid expense
batch_1 = airbnb_sample.sort_index().iloc[0:1000]

for i in batch_1:
    print(i)

In [12]:
batch_1['address_response'] = batch_1['lat_long'].apply(lambda x: get_address(x))
batch_1.head()

,latitude,longitude,room_type,accommodates,bedrooms,beds,price,minimum_nights,calendar_last_scraped,license,...,lock_on_bedroom_door,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,boroughs,frequencies,lat_long,address_response
0,40.68457,-73.91181,Private room,1,1.0,1.0,65.0,30,2023-10-02,NaN,...,0,0,1,0,0,0,Brooklyn,18101,"(40.68457, -73.91181)","{'address_components': [{'long_name': '983', '..."
1,40.76878,-73.98719,Private room,2,1.0,1.0,110.0,30,2023-10-02,NaN,...,0,0,0,1,0,0,Manhattan,23563,"(40.76878, -73.98719)","{'address_components': [{'long_name': '437', '..."
3,40.68180,-73.93121,Entire home/apt,2,1.0,1.0,70.0,45,2023-10-02,NaN,...,0,0,1,0,0,0,Brooklyn,18101,"(40.6818, -73.93121)","{'address_components': [{'long_name': '309', '..."
6,40.72813,-73.98801,Private room,1,1.0,1.0,125.0,1,2023-10-02,NaN,...,1,0,0,1,0,0,Manhattan,23563,"(40.72813, -73.98801)","{'address_components': [{'long_name': '120', '..."
12,40.66262,-73.74880,Private room,1,1.0,1.0,75.0,30,2023-10-02,NaN,...,1,0,0,0,1,0,Queens,7437,"(40.66262, -73.7488)",{'address_components': [{'long_name': '144-15'...


In [13]:
batch_1.to_csv(data_dir + 'airbnb_sample.csv')

In [18]:
batch_1['address_response'].apply(lambda x: len(x['address_components'])).sort_values(ascending=True)

3331     5
2717     6
2421     6
4816     7
2059     7
        ..
1987    10
3846    10
1746    10
3516    10
4765    10
Name: address_response, Length: 1000, dtype: int64

In [28]:
batch_1.loc[4765,'address_response']['address_components']

[{'long_name': '# 1', 'short_name': '# 1', 'types': ['subpremise']},
 {'long_name': '508', 'short_name': '508', 'types': ['street_number']},
 {'long_name': 'Hudson Street', 'short_name': 'Hudson St', 'types': ['route']},
 {'long_name': 'Manhattan',
  'short_name': 'Manhattan',
  'types': ['political', 'sublocality', 'sublocality_level_1']},
 {'long_name': 'New York',
  'short_name': 'New York',
  'types': ['locality', 'political']},
 {'long_name': 'New York County',
  'short_name': 'New York County',
  'types': ['administrative_area_level_2', 'political']},
 {'long_name': 'New York',
  'short_name': 'NY',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'United States',
  'short_name': 'US',
  'types': ['country', 'political']},
 {'long_name': '10014', 'short_name': '10014', 'types': ['postal_code']},
 {'long_name': '2854', 'short_name': '2854', 'types': ['postal_code_suffix']}]